In [1]:
import importlib
import DataPaths
import DatasetReader
import MeanMaker
importlib.reload(MeanMaker)
importlib.reload(DataPaths)
importlib.reload(DatasetReader)

import pandas as pd

In [2]:
mm = MeanMaker.MeanMaker()
dr = DatasetReader.DatasetReader()

In [3]:
full_year_contents = dr.read_directories(DataPaths.Paths.full_year_pickle_path)

In [4]:
full_year_contents

Directories: ['testfolder']
Files: ['full_year_2016_CarCount.pkl', 'full_year_2017_CarCount.pkl', 'full_year_2020_CarCount.pkl', 'full_year_2021_CarCount.pkl']
Full file paths: ['YOUR_PATH\\data\\full_year_pickles\\full_year_2016_CarCount.pkl', 'YOUR_PATH\\data\\full_year_pickles\\full_year_2017_CarCount.pkl', 'YOUR_PATH\\data\\full_year_pickles\\full_year_2020_CarCount.pkl', 'YOUR_PATH\\data\\full_year_pickles\\full_year_2021_CarCount.pkl']

In [5]:
all_means = []
for path in full_year_contents.full_file_paths:
  year = int(path.split("_")[-2])
  means_df = mm.create_means_from_monday(1, 6, year, 3, dr.read_pickle(path), "CarCount")
  all_means.append(means_df)

data = pd.concat(all_means)

[158, 159, 160, 161, 162, 163, 164]
[165, 166, 167, 168, 169, 170, 171]
[172, 173, 174, 175, 176, 177, 178]
[156, 157, 158, 159, 160, 161, 162]
[163, 164, 165, 166, 167, 168, 169]
[170, 171, 172, 173, 174, 175, 176]
[153, 154, 155, 156, 157, 158, 159]
[160, 161, 162, 163, 164, 165, 166]
[167, 168, 169, 170, 171, 172, 173]
[158, 159, 160, 161, 162, 163, 164]
[165, 166, 167, 168, 169, 170, 171]
[172, 173, 174, 175, 176, 177, 178]


In [6]:
data

,lamId,year,startDay,weeksInMean,direction,CarCount
0,1,2016,158,3,1,"[14.666666666666666, 9.666666666666666, 13.0, ..."
1,1,2016,158,3,2,"[16.0, 10.666666666666666, 14.0, 11.3333333333..."
2,99,2016,158,3,1,"[16.0, 16.333333333333332, 9.666666666666666, ..."
3,99,2016,158,3,2,"[26.333333333333332, 28.333333333333332, 30.66..."
4,102,2016,158,3,1,"[9.666666666666666, 12.0, 11.333333333333334, ..."
...,...,...,...,...,...,...
817,1604,2021,158,3,2,"[14.333333333333334, 8.0, 4.0, 9.0, 6.33333333..."
818,1605,2021,158,3,1,"[12.666666666666666, 7.666666666666667, 8.0, 5..."
819,1605,2021,158,3,2,"[17.0, 12.0, 7.333333333333333, 4.0, 10.333333..."
820,1606,2021,158,3,1,"[12.0, 6.666666666666667, 8.0, 6.3333333333333..."


In [7]:
lam_1 = data[data["lamId"]==3]

In [8]:
lam_1_dir_1 = lam_1[lam_1["direction"]==2]

In [9]:
lam_1_dir_1

,lamId,year,startDay,weeksInMean,direction,CarCount
3,3,2017,156,3,2,"[13.0, 12.0, 11.0, 8.333333333333334, 4.0, 7.3..."
3,3,2021,158,3,2,"[20.0, 16.0, 13.0, 11.0, 17.0, 12.0, 11.0, 14...."


In [10]:
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def blur(data, blur_range):
  data_len = len(data)
  blurred = []
  for i in range(data_len):
    sum = 0
    count = 0
    for j in range(i-1, i+blur_range+1):
      if j < 0 or j > data_len-1:
        continue
      sum += data[j]
      count += 1
    blurred.append(sum / count)
  return blurred


In [12]:
%matplotlib qt 
blur_amount = 20
hours = np.array([i for i in range(2016)]) / 12
# plt.plot(hours, blur(lam_1_dir_1["CarCount"].values[0], blur_amount), label="2016")
plt.plot(hours, blur(lam_1_dir_1["CarCount"].values[0], blur_amount), label="2017")
# plt.plot(hours, blur(lam_1_dir_1["CarCount"].values[2], blur_amount), label="2020")
plt.plot(hours, blur(lam_1_dir_1["CarCount"].values[1], blur_amount), label="2021")
plt.legend()

In [135]:
filtered = data.groupby(["lamId", "direction"]).filter(lambda x: len(x) == 4)

In [136]:
def vis_4years(lam_id, direction):
  vis_data = filtered[filtered["lamId"] == lam_id]
  vis_data = vis_data[vis_data["direction"] == direction]

  blur_amount = 20
  hours = np.array([i for i in range(2016)]) / 12
  plt.plot(hours, blur(vis_data["CarCount"].values[0], blur_amount), label="2016")
  plt.plot(hours, blur(vis_data["CarCount"].values[1], blur_amount), label="2017")
  plt.plot(hours, blur(vis_data["CarCount"].values[2], blur_amount), label="2020")
  plt.plot(hours, blur(vis_data["CarCount"].values[3], blur_amount), label="2021")
  plt.legend()

In [138]:
filtered["lamId"].unique()

array([   1,   99,  104,  105,  106,  107,  108,  109,  110,  111,  112,
        114,  115,  116,  117,  119,  121,  122,  123,  124,  127,  129,
        131,  133,  134,  135,  138,  139,  141,  142,  143,  144,  145,
        146,  147,  149,  150,  151,  152,  153,  155,  158,  159,  160,
        162,  163,  164,  165,  169,  172,  175,  176,  177,  178,  182,
        183,  184,  185,  186,  187,  188,  190,  191,  192,  193,  194,
        195,  196,  197,  198,  201,  203,  204,  205,  206,  207,  208,
        209,  210,  211,  221,  223,  224,  225,  226,  227,  231,  232,
        233,  234,  235,  237,  239,  242,  243,  244,  247,  248,  249,
        251,  252,  254,  255,  257,  302,  303,  304,  305,  306,  401,
        402,  403,  404,  405,  406,  407,  408,  409,  421,  422,  424,
        425,  426,  428,  430,  431,  433,  435,  436,  437,  438,  439,
        440,  442,  445,  447,  448,  449,  450,  451,  453,  455,  456,
        458,  460,  461,  462,  463,  464,  465,  4

In [147]:
vis_4years(110, 2)

In [148]:
pre_post_covid_sulku = []

all_means.append(mm.create_means_from_monday(1, 3, 2020, 3, dr.read_pickle('YOUR_PATH\\data\\full_year_pickles\\full_year_2020_CarCount.pkl'), "CarCount"))
all_means.append(mm.create_means_from_monday(1, 4, 2020, 3, dr.read_pickle('YOUR_PATH\\data\\full_year_pickles\\full_year_2020_CarCount.pkl'), "CarCount"))

covid = pd.concat(all_means)

[62, 63, 64, 65, 66, 67, 68]
[69, 70, 71, 72, 73, 74, 75]
[76, 77, 78, 79, 80, 81, 82]
[97, 98, 99, 100, 101, 102, 103]
[104, 105, 106, 107, 108, 109, 110]
[111, 112, 113, 114, 115, 116, 117]


In [149]:
covid

,lamId,year,startDay,weeksInMean,direction,CarCount
0,1,2016,158,3,1,"[14.666666666666666, 9.666666666666666, 13.0, ..."
1,1,2016,158,3,2,"[16.0, 10.666666666666666, 14.0, 11.3333333333..."
2,99,2016,158,3,1,"[16.0, 16.333333333333332, 9.666666666666666, ..."
3,99,2016,158,3,2,"[26.333333333333332, 28.333333333333332, 30.66..."
4,102,2016,158,3,1,"[9.666666666666666, 12.0, 11.333333333333334, ..."
...,...,...,...,...,...,...
835,1605,2020,97,3,2,"[0.0, 1.3333333333333333, 0.3333333333333333, ..."
836,1606,2020,97,3,1,"[5.0, 2.3333333333333335, 0.6666666666666666, ..."
837,1606,2020,97,3,2,"[1.0, 0.3333333333333333, 2.3333333333333335, ..."
838,1607,2020,97,3,1,"[0.6666666666666666, 0.6666666666666666, 5.333..."


In [150]:
pre = covid[covid["startDay"] == 62]
post = covid[covid["startDay"] == 97]

In [201]:
def vis_pre_and_post_covid(lam_id, direction, blur_amount):

  pre_y = pre[pre["lamId"]==lam_id]
  pre_y = pre_y[pre_y["direction"]==direction]["CarCount"].values[0]
  post_y = post[post["lamId"]==lam_id]
  post_y = post_y[post_y["direction"]==direction]["CarCount"].values[0]
  pre_y = blur(pre_y, blur_amount)
  post_y = blur(post_y, blur_amount)
  plt.plot(pre_y, label="Pre")
  plt.plot(post_y, label="Post")
  diff = []
  for i in range(len(post_y)):
    diff.append(post_y[i]-pre_y[i])
  plt.plot(diff, label="diff")
  plt.legend()

In [202]:
pre["lamId"].unique()

array([   1,    3,    4,    5,    6,    7,    8,    9,   11,   98,   99,
        102,  103,  104,  105,  106,  107,  108,  109,  110,  111,  112,
        114,  115,  116,  117,  118,  119,  121,  122,  123,  124,  125,
        126,  127,  128,  129,  130,  131,  133,  134,  135,  136,  137,
        138,  139,  140,  141,  142,  143,  144,  145,  146,  147,  148,
        149,  150,  151,  152,  153,  154,  155,  156,  158,  159,  160,
        162,  163,  165,  167,  168,  169,  172,  175,  176,  177,  178,
        179,  182,  183,  184,  185,  186,  187,  188,  189,  190,  191,
        192,  193,  194,  195,  196,  197,  198,  201,  203,  204,  205,
        206,  207,  208,  209,  210,  211,  221,  223,  224,  225,  226,
        227,  230,  231,  232,  233,  234,  235,  237,  238,  239,  240,
        242,  243,  244,  245,  247,  248,  249,  250,  251,  252,  254,
        257,  302,  303,  304,  305,  306,  307,  308,  309,  310,  401,
        402,  403,  404,  405,  406,  407,  408,  4

In [217]:
vis_pre_and_post_covid(130, 1, 25)

array([array([3.        , 6.66666667, 4.33333333, ..., 2.33333333, 4.33333333,
              7.66666667])                                                    ],
      dtype=object)

In [226]:
fft_data = pre.iloc[6]["CarCount"]


In [231]:
plt.plot(fftd)

C:\Users\Mikko\anaconda3\envs\gradu\lib\site-packages\numpy\core\_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [229]:
fftd = np.fft.fft(fft_data)

array([68411.33333333   +0.j        , -2479.61563364-4560.20909429j,
        1410.23295417-2494.89864111j, ...,  2169.37764669 +325.01314138j,
        1410.23295417+2494.89864111j, -2479.61563364+4560.20909429j])